In [1]:
import source.prepare_data as prd
import source.dataset as dat
import source.multi_layer_perceptron as mlp
import source.table as tb
import source.comparision as comp
import source.recurent_neural_networks as rnn

C:\Python_files\DL_final2\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
import spacy
from importlib import reload

In [14]:
reload(prd)
reload(dat)
reload(mlp)
reload(tb)
reload(comp)

<module 'source.comparision' from 'C:\\Python_files\\DL_final2\\source\\comparision.py'>

In [3]:
"""
1) Preprocessing
"""
# load models and dataset
df = pd.read_csv("hf://datasets/gxb912/large-twitter-tweets-sentiment/train.csv", nrows=1000)
nlp = spacy.load("en_core_web_sm")

# prepare dataset
df = prd.prepare_df(df, nlp)

In [7]:
# create dataset out of cleaned columns
dataset = dat.Text_Dataset(df, col_text="clean_text_str", col_label="sentiment", args={"max_features":5000})
dataset.split_dataset()

In [15]:
"""
2) Accuracy analysis for MLP model. We will compare the workflow of MLP by using TD-IDF encoding and BOW encoding

2.1) TD-IDF encoding
"""
X_train_TF, X_test_TF, Y_train_TF, Y_test_TF = dataset.get_encodings(tdidf=True)

mlp_tdidf_params = mlp.find_best_mlp(X_train=X_train_TF,
                                      Y_train=Y_train_TF,
                                      X_test=X_test_TF,
                                      Y_test=Y_test_TF,
                                      n_trials = 3,
                                     encoding="TDIFD")

[I 2025-06-15 20:50:41,968] A new study created in memory with name: mlp_optimization
[I 2025-06-15 20:50:44,365] Trial 0 finished with values: [0.5604890584945679, 0.7699999809265137] and parameters: {'epochs': 172, 'learning_rate': 0.19215569438181865, 'units': 2}.
[I 2025-06-15 20:50:47,019] Trial 1 finished with values: [0.5489175915718079, 0.7350000143051147] and parameters: {'epochs': 195, 'learning_rate': 0.12237232274459567, 'units': 5}.
[I 2025-06-15 20:50:49,486] Trial 2 finished with values: [0.7252979874610901, 0.7350000143051147] and parameters: {'epochs': 171, 'learning_rate': 0.43412688838275926, 'units': 10}.


In [16]:
mlp_tdidf_params

,loss,accuracy,params_epochs,params_learning_rate,params_units,time,name
0,0.548918,0.735,195,0.122372,5,2.653508,MLP_BOW
1,0.725298,0.735,171,0.434127,10,2.466822,MLP_BOW
2,0.560489,0.770,172,0.192156,2,2.395507,MLP_BOW


In [18]:
"""
2.2) BOW encoding
"""
X_train_TF, X_test_TF, Y_train_TF, Y_test_TF = dataset.get_encodings(tdidf=False)

mlp_bow_params = mlp.find_best_mlp(X_train=X_train_TF,
                                      Y_train=Y_train_TF,
                                      X_test=X_test_TF,
                                      Y_test=Y_test_TF,
                                      n_trials = 3,
                                     encoding="BOW")

[I 2025-06-15 20:55:24,721] A new study created in memory with name: mlp_optimization
[I 2025-06-15 20:55:26,574] Trial 0 finished with values: [0.6566829681396484, 0.5799999833106995] and parameters: {'epochs': 110, 'learning_rate': 0.015252602040205392, 'units': 5}.
[I 2025-06-15 20:55:28,375] Trial 1 finished with values: [0.783537745475769, 0.75] and parameters: {'epochs': 107, 'learning_rate': 0.23558898005821538, 'units': 2}.
[I 2025-06-15 20:55:30,514] Trial 2 finished with values: [0.5722466707229614, 0.7099999785423279] and parameters: {'epochs': 126, 'learning_rate': 0.04440677561981615, 'units': 4}.


In [19]:
mlp_bow_params

,loss,accuracy,params_epochs,params_learning_rate,params_units,time,name
0,0.656683,0.58,110,0.015253,5,1.851849,MLP_BOW
1,0.572247,0.71,126,0.044407,4,2.139020,MLP_BOW
2,0.783538,0.75,107,0.235589,2,1.800016,MLP_BOW


In [ ]:
"""
3) Accuracy analysis for RNN models.
3.1) Classical RNN.
"""
